<a href="https://colab.research.google.com/github/CamiloVga/Prueba-Tecnica_Bid/blob/main/Ejercicio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 2

Para abordar el desafío de implementar un sistema RAG (Retrieval Augmented Generation) enfocado en documentos fiscales del BID, se ha desarrollado una solución integral que combina tecnologías de vanguardia en procesamiento de lenguaje natural. El sistema se construye sobre langchain como framework principal, aprovechando el potente modelo de lenguaje Llama-2-7b-chat-hf de HuggingFace junto con embeddings multilingual-e5-large, lo que asegura una comprensión profunda de documentos técnicos fiscales en múltiples idiomas.

La estretegia para el ejercicio se explica en detalle en este script. Pero pueden ver el producto final en esta app web diseñada para brindar una buena experiencia de usuario: https://huggingface.co/spaces/CamiloVega/Fislac_Bot


**Recomendaciones de uso del script**

1. El script está diseñado para ejecutarse en Google Colab, lo que elimina la necesidad de configuraciones locales complejas.

2. Requisitos importantes:
   - Se necesita una cuenta en Hugging Face
   - Requerirá un token de acceso de Hugging Face y guardarlo como un secreto en Google Colab
   - Se recomienda tener acceso a modelos de Llama 2 en Hugging Face

3. Archivos de prueba:
   - El sistema espera documentos PDF sobre FISLAC y documentos técnicos del BID
   - Los documentos se clasifican automáticamente como técnicos o Q&A
   - Los archivos deben subirse cuando aparezca el widget de carga en Colab

4. Recursos computacionales:
   - Se requiere un runtime con GPU (T4 o mejor)
   - Memoria RAM recomendada: 12GB o superior
   - El modelo se carga en 8-bit para optimizar memoria



# 1. Configuraciones iniciales
La implementación de nuestro sistema RAG se fundamenta en una cuidadosa selección de componentes tecnológicos que trabajan en armonía. Como base fundamental, integramos PyPDF2 y pdfplumber para garantizar un procesamiento robusto de documentos PDF, mientras que langchain-community actúa como el framework principal, proporcionando una arquitectura modular esencial para construir aplicaciones de IA contextual.

Esta elección es particularmente valiosa al trabajar con documentos técnicos del BID, donde la precisión y el mantenimiento del contexto financiero son cruciales. Para la gestión eficiente de vectores, incorporamos FAISS, destacando por su capacidad excepcional en indexación y búsqueda vectorial de alta dimensionalidad, lo que permite una recuperación semántica veloz y precisa en documentos fiscales extensos.

El núcleo del sistema se potencia con transformers y sentence-transformers de Hugging Face, elegidos por su sobresaliente soporte multilingüe, una característica indispensable para manejar la documentación del BID en español e inglés. La arquitectura se complementa con un sistema robusto de logging para monitorear el procesamiento de documentos y typing para mantener la integridad del código.

Es destacable la implementación del modelo multilingual-e5-large, que supera a alternativas anteriores como flan-t5-base y paraphrase-multilingual-MiniLM-L12-v2, ofreciendo una interpretación más precisa y contextualizada de conceptos fiscales en múltiples idiomas.

In [ ]:
# Instalación de dependencias necesarias
!pip install pypdf
!pip install PyPDF2 pdfplumber
!pip install langchain-community
!pip install faiss-cpu
!pip install transformers
!pip install sentence-transformers
!pip install torch
!pip install gradio
!pip install accelerate
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#Importaciones

import os
import logging
from typing import List, Dict, Optional
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
# Importaciones de transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline
)
from langchain.llms import HuggingFacePipeline
from google.colab import files
from langchain_community.document_loaders import PyPDFLoader

# Configuración de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)




## 1.2 Base de conocimiento
En esta parte se prepara el directorio de la base de conocimiento y se abre el widget para subir los documentos en PDF sobre los cuales se quiera aplicar el sistema RAG

In [ ]:
# Limpiar y crear directorio para la base de conocimiento
!rm -rf knowledge_base
!mkdir knowledge_base

print("Por favor, sube tus archivos PDF cuando aparezca el botón...")
uploaded = files.upload()

# Guardar los archivos en el directorio knowledge_base
for filename, content in uploaded.items():
    if filename.endswith('.pdf'):
        path = f'knowledge_base/{filename}'
        with open(path, 'wb') as f:
            f.write(content)
        print(f"Archivo {filename} guardado en knowledge_base/")

# Verificar archivos subidos
!ls knowledge_base

Por favor, sube tus archivos PDF cuando aparezca el botón...


Saving 22. Valencia, Díaz, Parra (2022) - Assessing macro-fiscal risk for Latin American and Caribbean countries.pdf to 22. Valencia, Díaz, Parra (2022) - Assessing macro-fiscal risk for Latin American and Caribbean countries.pdf
Saving fislac_kb.pdf to fislac_kb.pdf
Archivo 22. Valencia, Díaz, Parra (2022) - Assessing macro-fiscal risk for Latin American and Caribbean countries.pdf guardado en knowledge_base/
Archivo fislac_kb.pdf guardado en knowledge_base/
'22. Valencia, Díaz, Parra (2022) - Assessing macro-fiscal risk for Latin American and Caribbean countries.pdf'
 fislac_kb.pdf


# 2. Estructuración del sistema RAG

El sistema RAG implementa una arquitectura secuencial sofisticada para procesar y responder consultas sobre documentos fiscales del BID, adaptándose dinámicamente al tipo de contenido. En la primera fase, DocumentLoader realiza una ingesta inteligente de PDFs, enriqueciendo los metadatos para distinguir entre documentos técnicos y documentos tipo Q&A, facilitando así un procesamiento más preciso.

TextProcessor implementa una estrategia de segmentación dual: para documentos técnicos, utiliza chunks de 800 tokens con una superposición de 200 tokens, permitiendo mantener el contexto en análisis complejos, mientras que para documentos tipo Q&A emplea chunks más compactos de 500 tokens con una superposición de 100 tokens, optimizando la recuperación de información puntual. La fase de indexación aprovecha el poder del modelo multilingual-e5-large para generar embeddings de alta calidad, que FAISS organiza eficientemente en su estructura vectorial, facilitando búsquedas semánticas precisas.

El pipeline culmina con un sistema de recuperación contextual mejorado que selecciona los 6 fragmentos más relevantes, alimentándolos al modelo Llama-2-7b-chat-hf junto con un prompt template especializado en español. Esta configuración asegura respuestas coherentes y precisas, manteniendo el contexto técnico-fiscal necesario mientras adapta su estrategia según el tipo de documento y consulta.

In [ ]:
#Clases del Sistema

class DocumentLoader:
    """Clase para gestionar la carga de documentos PDF."""

    @staticmethod
    def load_pdfs(directory_path: str) -> List:
        documents = []
        pdf_files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]

        for pdf_file in pdf_files:
            pdf_path = os.path.join(directory_path, pdf_file)
            try:
                loader = PyPDFLoader(pdf_path)
                pdf_documents = loader.load()

                # Metadatos mejorados para contextualización
                for doc in pdf_documents:
                    doc.metadata.update({
                        'title': pdf_file,
                        'type': 'technical' if 'Valencia' in pdf_file else 'qa',
                        'language': 'es',
                        'page': doc.metadata.get('page', 0)
                    })
                    documents.append(doc)

                logger.info(f"Documento {pdf_file} cargado exitosamente")
            except Exception as e:
                logger.error(f"Error al cargar {pdf_file}: {str(e)}")

        return documents

class TextProcessor:
    """Clase para procesar y dividir el texto en chunks."""

    def __init__(self):
        self.technical_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800,
            chunk_overlap=200,
            separators=["\n\n", "\n", ". ", " ", ""],
            length_function=len,
            is_separator_regex=False
        )

        self.qa_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=100,
            separators=["\n\n", "\n", ". ", " ", ""],
            length_function=len,
            is_separator_regex=False
        )

    def process_documents(self, documents: List) -> List:
        """Procesa los documentos con configuración específica según tipo."""
        if not documents:
            logger.warning("No hay documentos para procesar")
            return []

        processed_chunks = []
        for doc in documents:
            # Seleccionar el splitter según el tipo de documento
            splitter = self.technical_splitter if doc.metadata['type'] == 'technical' else self.qa_splitter

            chunks = splitter.split_documents([doc])
            processed_chunks.extend(chunks)

        logger.info(f"Documentos procesados en {len(processed_chunks)} fragmentos")
        return processed_chunks

class RAGSystem:
    """Sistema principal de RAG con configuración optimizada."""

    def __init__(self, model_name: str = "meta-llama/Llama-2-7b-chat-hf"):
        self.model_name = model_name
        self.embeddings = None
        self.vector_store = None
        self.qa_chain = None
        self.tokenizer = None
        self.model = None

    def initialize_embeddings(self):
        """Inicializa embeddings con modelo multilingüe optimizado."""
        try:
            self.embeddings = HuggingFaceEmbeddings(
                model_name="intfloat/multilingual-e5-large",
                model_kwargs={'device': 'cuda'},
                encode_kwargs={'normalize_embeddings': True}
            )
            logger.info("Modelo de embeddings inicializado exitosamente")
        except Exception as e:
            logger.error(f"Error al inicializar embeddings: {str(e)}")
            raise

    def create_vector_store(self, processed_chunks: List):
        """Crea almacén de vectores con configuración mejorada."""
        if not self.embeddings:
            raise ValueError("Embeddings no inicializados. Ejecute initialize_embeddings primero.")

        try:
            self.vector_store = FAISS.from_documents(
                processed_chunks,
                self.embeddings
            )
            logger.info("Almacén de vectores creado exitosamente")
        except Exception as e:
            logger.error(f"Error al crear vector store: {str(e)}")
            raise

    def setup_rag_pipeline(self):
        """Configura pipeline RAG con prompt mejorado."""
        if not self.vector_store:
            raise ValueError("Vector store no inicializado")

        try:
            # Configuración del modelo
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                load_in_8bit=True,
                device_map="auto",
                trust_remote_code=True
            )

            # Pipeline de generación optimizado
            pipe = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                max_length=2048,
                temperature=0.1,
                top_p=0.95,
                repetition_penalty=1.15,
                truncation=True
            )

            llm = HuggingFacePipeline(pipeline=pipe)

            # Prompt template simplificado
            prompt_template = """
            Contexto: {context}

            Basándote en el contexto anterior, responde a la siguiente pregunta de manera clara y concisa en español.
            Si la información no está en el contexto, indícalo explícitamente.

            Pregunta: {question}
            """

            PROMPT = PromptTemplate(
                template=prompt_template,
                input_variables=["context", "question"]
            )

            # Configuración de la cadena QA
            self.qa_chain = RetrievalQA.from_chain_type(
                llm=llm,
                chain_type="stuff",
                retriever=self.vector_store.as_retriever(
                    search_kwargs={"k": 6, "score_threshold": 0.5}
                ),
                return_source_documents=True,
                chain_type_kwargs={"prompt": PROMPT}
            )

            logger.info("Pipeline RAG configurado exitosamente")
        except Exception as e:
            logger.error(f"Error al configurar pipeline RAG: {str(e)}")
            raise

    def answer_question(self, question: str) -> Dict:
        """Procesa una pregunta y genera una respuesta basada en el contexto recuperado."""
        if not self.qa_chain:
            raise ValueError("Pipeline RAG no configurado. Ejecute setup_rag_pipeline primero.")

        try:
            # Procesar la pregunta
            result = self.qa_chain({"query": question})

            # Estructurar la respuesta
            response = {
                'answer': result['result'],
                'sources': []
            }

            # Procesar documentos fuente
            for doc in result['source_documents']:
                source = {
                    'title': doc.metadata.get('title', 'Unknown'),
                    'content': doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content,
                    'metadata': doc.metadata,
                    'score': doc.metadata.get('score', 0.0)
                }
                response['sources'].append(source)

            return response

        except Exception as e:
            logger.error(f"Error al procesar la pregunta: {str(e)}")
            raise

def test_rag_system(question: str):
    """Función que muestra solo pregunta, respuesta y fuentes."""
    try:
        response = rag_system.answer_question(question)

        # Extraer solo la respuesta final después de "Respuesta:"
        answer = response['answer']
        if "Respuesta:" in answer:
            answer = answer.split("Respuesta:")[-1].strip()
        elif "Respuesta (en español):" in answer:
            answer = answer.split("Respuesta (en español):")[-1].strip()

        # Eliminar todo el texto antes de la respuesta real
        if "Basándote en el contexto anterior" in answer:
            answer = answer.split("Basándote en el contexto anterior")[0].strip()

        # Imprimir solo la información esencial
        print(f"Pregunta: {question}")
        print(f"Respuesta: {answer}\n")
        print("Fuentes utilizadas:")

        # Imprimir solo las fuentes relevantes
        for source in response['sources'][:3]:  # Limitamos a 3 fuentes
            doc_title = source['title']
            doc_type = source['metadata']['type']
            excerpt = source['content'][:100] + "..." if len(source['content']) > 100 else source['content']
            print(f"- {doc_title} ({doc_type})")
            print(f"  {excerpt}\n")

        print("-" * 80 + "\n")  # Separador visual

    except Exception as e:
        print(f"Error al procesar la pregunta: {str(e)}")

# 2.2 Estructuración del sistema RAG

En esta parte se implementa una secuencia cuidadosamente orquestada para inicializar y configurar el pipeline RAG. El proceso comienza verificando la existencia de documentos PDF en el directorio knowledge_base mediante una comprensión de lista, garantizando una base documental adecuada para el procesamiento.

La fase de carga utiliza DocumentLoader para procesar los PDFs, enriqueciendo automáticamente los metadatos con información crucial como el tipo de documento (técnico o Q&A) y el idioma. TextProcessor aplica entonces su estrategia de segmentación adaptativa, utilizando diferentes configuraciones de chunks según la naturaleza del documento: chunks más extensos para documentos técnicos y más compactos para documentos tipo Q&A.

El sistema RAG se inicializa en tres pasos secuenciales críticos: primero, se configuran los embeddings utilizando el modelo multilingual-e5-large optimizado para GPU; segundo, se construye el vector store con FAISS utilizando los chunks procesados; y finalmente, se configura el pipeline completo con Llama-2-7b-chat-hf, incluyendo optimizaciones de memoria mediante cuantización de 8-bits. El sistema proporciona feedback detallado en cada etapa, monitoreando la cantidad de documentos cargados, el número de chunks generados y confirmando la configuración exitosa de cada componente del pipeline RAG.

In [ ]:
# Inicialización y configuración del sistema
print("Verificando documentos en el directorio...")
pdf_files = [f for f in os.listdir("knowledge_base") if f.endswith('.pdf')]
if not pdf_files:
    raise ValueError("No hay archivos PDF en knowledge_base. Por favor, sube algunos archivos primero.")

print(f"Encontrados {len(pdf_files)} archivos PDF")

# Cargar documentos
loader = DocumentLoader()
documents = loader.load_pdfs("knowledge_base")

if not documents:
    raise ValueError("No se pudieron cargar los documentos")

print(f"Se cargaron {len(documents)} documentos")

# Procesar documentos
processor = TextProcessor()
processed_chunks = processor.process_documents(documents)

print(f"Se crearon {len(processed_chunks)} chunks")

# Configurar sistema RAG
rag_system = RAGSystem()
rag_system.initialize_embeddings()
rag_system.create_vector_store(processed_chunks)
rag_system.setup_rag_pipeline()

print("Sistema RAG creado exitosamente")

Verificando documentos en el directorio...
Encontrados 2 archivos PDF
Se cargaron 47 documentos
Se crearon 216 chunks


<ipython-input-4-fe8c5e958fba>:84: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Sistema RAG creado exitosamente


<ipython-input-4-fe8c5e958fba>:136: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


# 3. Pruebas del sistema
La fase de validación del sistema RAG se implementa a través de una función de prueba estructurada y ampliada que evalúa múltiples aspectos del procesamiento de información fiscal. La función test_rag_system sirve como interfaz unificada que no solo procesa las consultas, sino que también proporciona una visión detallada de las fuentes utilizadas, incluyendo el tipo de documento, la página específica y la confianza de la recuperación.

Para una validación sistemática y exhaustiva, el sistema incluye un conjunto diversificado de preguntas de prueba que abarcan tres categorías principales: preguntas sobre el contenido técnico del paper de Valencia et al. (como variables macroeconómicas relevantes y comparativas entre países), consultas sobre la plataforma FISLAC (incluyendo sus módulos y funcionalidades), y preguntas que requieren la integración de información de ambas fuentes.

La función procesa cada consulta mostrando la respuesta generada junto con metadatos enriquecidos de las fuentes utilizadas, permitiendo evaluar no solo la precisión de las respuestas sino también la capacidad del sistema para mantener el contexto técnico y relacionar información de diferentes documentos. Se ha optimizado el manejo de respuestas para eliminar artefactos del prompt y garantizar una presentación clara y profesional de la información, manteniendo la trazabilidad completa de las fuentes consultadas.

In [ ]:
def test_rag_system(question: str):
    """Función mejorada para probar el sistema RAG."""
    print(f"\nPregunta: {question}")

    try:
        response = rag_system.answer_question(question)

        print("\nRespuesta:", response['answer'])
        print("\nFuentes utilizadas:")
        for source in response['sources']:
            print(f"- Documento: {source['title']} (Tipo: {source['metadata']['type']})")
            print(f"  Extracto: {source['content']}")
            print(f"  Confianza: {source['score']:.2f}")
        print("\n" + "="*50)
    except Exception as e:
        print(f"Error al procesar la pregunta: {str(e)}")

# Lista de preguntas de prueba
test_questions = [
    "¿Qué es FISLAC?"
]

print("Ejecutando pruebas con preguntas diversas...")
for question in test_questions:
    test_rag_system(question)

Ejecutando pruebas con preguntas diversas...

Pregunta: ¿Qué es FISLAC?


<ipython-input-4-fe8c5e958fba>:176: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.qa_chain({"query": question})



Respuesta: 
            Contexto: What is FISLAC? 
FISLAC (Fiscal Sustainability for Latin American and the Caribbean countries) is an 
ecosystem developed by the Inter-American Development Bank to help governments of 
Latin American and Caribbean countries (LAC) strengthen macro -fiscal policy and 
decision-making. 
FISLAC is comprised of two primary elements: the models and the website. The models 
serve as the core of the tools displayed on the website. Upon exploration, within the

serve as the core of the tools displayed on the website. Upon exploration, within the 
website you will discover a variety of dashboards and tools designed to facilitate a 
comprehensive understanding of the macro-fiscal dynamics of LAC countries. 
Why FISLAC? 
FISLAC is an agile and accessible platform that provides an overview of the macro-fiscal 
risks faced by countries in the LAC region. 
How can I access FISLAC? 
To access the FISLAC tool, go to https://www.fislac.com

FISLAC-EWS is a web-app that

#3.1 Inferencia por medio de preguntas interactivas

En esta parte se implementa una interfaz interactiva que permite evaluar el sistema RAG en tiempo real. La función ask_custom_question() establece un bucle de interacción continua donde el usuario puede realizar consultas libres sobre documentos fiscales del BID.

El sistema incluye un mecanismo de salida controlada ('q' para terminar) y procesamiento de input (strip()) para manejar espacios extra. Cada pregunta se procesa a través del mismo pipeline RAG usado en las pruebas sistemáticas, manteniendo consistencia en el procesamiento y formato de respuestas. Los warnings mostrados son informativos sobre la configuración del modelo y no afectan la funcionalidad del sistema.

In [ ]:
#Función interactiva
def ask_custom_question():
    while True:
        question = input("\nEscribe tu pregunta (o 'q' para salir): ").strip()
        if question.lower() == 'q':
            break
        test_rag_system(question)

print("\nAhora puedes hacer tus propias preguntas:")
ask_custom_question()


Ahora puedes hacer tus propias preguntas:

Escribe tu pregunta (o 'q' para salir): qué modulos tiene fislac?

Pregunta: qué modulos tiene fislac?

Respuesta: 
            Contexto: How can I access FISLAC? 
To access the FISLAC tool, go to https://www.fislac.com  
What can I find in FISLAC? 
La página web de FISLAC cuenta con varios módulos: 
1. THE CLIMATE CHANGE DASHBOARD: Presents a Deep Dive into the Economic and 
Fiscal Implications of Environmental Challenges in LAC 
2. PUBLIC INVESTMENT DASHBOARD : Dashboard that all ows to explor the  Public 
Investment in Latin America.  
3. FISCAL RULES COMPLIANCE HUB: A Hub for Transparency and Accountability in

What is FISLAC? 
FISLAC (Fiscal Sustainability for Latin American and the Caribbean countries) is an 
ecosystem developed by the Inter-American Development Bank to help governments of 
Latin American and Caribbean countries (LAC) strengthen macro -fiscal policy and 
decision-making. 
FISLAC is comprised of two primary elements: the

## 3.2 Inferencia tipo chatbot
Para mejorar la experiencia de usuario se propone usar el framework de Gradio para brindar una experiencia RAG Chatbot.

Al correr el código se abre una interfaz dentro de este script de Google Colab para usar el chatbot, o también se puede utilizar en una ventana independiente del navegador danto clic en la url que termina en gradio.live

In [ ]:
import gradio as gr

def create_chatbot_interface(rag_system):
    """Crea una interfaz de chatbot mejorada con botones nativos de Gradio."""

    with gr.Blocks(css="""
        div.gradio-container {background-color: #f0f2f6}
        div.message {font-size: 16px}
        div.contain {max-width: 1200px; margin: auto}
        div.message-wrap {width: 80%}
    """) as demo:
        # Header
        gr.HTML("""
            <div style="text-align: center; max-width: 1200px; margin: 0 auto; padding: 20px;">
                <h1 style="color: #2d333a;">📊 Fislac Bot</h1>
                <p style="color: #4a5568;">
                    🤖 Asistente especializado en análisis fiscal y documentación de FISLAC
                </p>
            </div>
        """)

        chatbot = gr.Chatbot(
            show_label=False,
            container=True,
            height=500,
            bubble_full_width=True,
            show_copy_button=True,
            scale=2
        )

        with gr.Row():
            message = gr.Textbox(
                placeholder="💭 Escribe tu pregunta aquí...",
                show_label=False,
                container=False,
                scale=8,
                autofocus=True
            )
            clear = gr.Button("🗑️ Limpiar", size="sm", scale=1)

        # Preguntas sugeridas usando componentes nativos de Gradio
        gr.HTML('<p style="color: #2d333a; font-weight: bold; margin: 20px 0 10px 0;">💡 Preguntas sugeridas:</p>')
        with gr.Row():
            sugerencia1 = gr.Button("¿Qué es FISLAC?", scale=1)
            sugerencia2 = gr.Button("¿Cuáles son los módulos principales de FISLAC?", scale=1)

        with gr.Row():
            sugerencia3 = gr.Button("¿Qué variables macroeconómicas son relevantes para economías avanzadas?", scale=1)
            sugerencia4 = gr.Button("¿Cómo se compara el riesgo fiscal entre países emergentes y avanzados?", scale=1)

        # Footer
        gr.HTML("""
            <div style="text-align: center; max-width: 1200px; margin: 20px auto; padding: 20px;
                        background-color: #f8f9fa; border-radius: 10px;">
                <div style="margin-bottom: 15px;">
                    <h3 style="color: #2d333a;">🔍 Acerca de este asistente</h3>
                    <p style="color: #666; font-size: 14px;">
                        Este bot utiliza tecnología RAG (Retrieval Augmented Generation) que combina:
                    </p>
                    <ul style="list-style: none; color: #666; font-size: 14px;">
                        <li>🔹 Motor LLM: Llama-2-7b-chat-hf</li>
                        <li>🔹 Embeddings: multilingual-e5-large</li>
                        <li>🔹 Vector Store: FAISS</li>
                    </ul>
                </div>
                <div style="border-top: 1px solid #ddd; padding-top: 15px;">
                    <p style="color: #666; font-size: 14px;">
                        <strong>Base de conocimiento actual:</strong><br>
                        • Valencia et al. (2022) - "Assessing macro-fiscal risk for Latin American and Caribbean countries"<br>
                        • Documentación técnica de FISLAC
                    </p>
                </div>
                <div style="border-top: 1px solid #ddd; margin-top: 15px; padding-top: 15px;">
                    <p style="color: #666; font-size: 12px; font-style: italic;">
                        Desarrollado usando LangChain. Sistema RAG implementado siguiendo las mejores prácticas
                        de LangChain para crear sistemas conversacionales basados en documentos.
                    </p>
                </div>
                <div style="border-top: 1px solid #ddd; margin-top: 15px; padding-top: 15px;">
                    <p style="color: #666; font-size: 14px;">
                        Desarrollado por <a href="https://www.linkedin.com/in/camilo-vega-169084b1/" target="_blank"
                        style="color: #2196F3; text-decoration: none; font-weight: bold;">Camilo Vega</a>,
                        Consultor en Inteligencia Artificial 🤖
                    </p>
                </div>
            </div>
        """)

        def respond(message, history):
            """Función para procesar respuestas con fuentes relevantes."""
            try:
                response = rag_system.answer_question(message)

                # Extraer solo la respuesta limpia
                answer = response['answer']
                if "Respuesta:" in answer:
                    answer = answer.split("Respuesta:")[-1].strip()
                elif "Respuesta (en español):" in answer:
                    answer = answer.split("Respuesta (en español):")[-1].strip()

                # Eliminar texto del prompt
                if "Basándote en el contexto anterior" in answer:
                    answer = answer.split("Basándote en el contexto anterior")[0].strip()

                # Traducir términos en inglés comunes
                answer = answer.replace("Macroeconomic stability variables", "Variables de estabilidad macroeconómica")
                answer = answer.replace("such as", "como")
                answer = answer.replace("and", "y")

                # Añadir las fuentes utilizadas
                if response.get('sources'):
                    # Tomar las primeras 3 fuentes que fueron usadas para generar la respuesta
                    relevant_sources = set([source['title'] for source in response['sources'][:3]])
                    if relevant_sources:
                        answer += "\n\n📚 Fuentes consultadas:\n" + "\n".join([f"• {source}" for source in relevant_sources])

                history.append((message, answer))
                return history

            except Exception as e:
                return history + [(message, f"❌ Lo siento, ocurrió un error: {str(e)}")]

        # Event handlers para las preguntas sugeridas
        def suggest_question(question):
            return respond(question, chatbot.value)

        # Configurar eventos
        message.submit(respond, [message, chatbot], [chatbot])
        clear.click(lambda: [], None, chatbot, queue=False)

        # Conectar botones de sugerencias
        sugerencia1.click(lambda: suggest_question("¿Qué es FISLAC?"), None, [chatbot])
        sugerencia2.click(lambda: suggest_question("¿Cuáles son los módulos principales de FISLAC?"), None, [chatbot])
        sugerencia3.click(lambda: suggest_question("¿Qué variables macroeconómicas son relevantes para economías avanzadas?"), None, [chatbot])
        sugerencia4.click(lambda: suggest_question("¿Cómo se compara el riesgo fiscal entre países emergentes y avanzados?"), None, [chatbot])

    return demo

demo = create_chatbot_interface(rag_system)
demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a2b85a8f4b4632ed2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
